In [ ]:
from urllib.request import build_opener, HTTPCookieProcessor
from bs4 import BeautifulSoup

import json
import csv

In [ ]:
# Create page opener with cookie
page_opener = build_opener(HTTPCookieProcessor())

In [ ]:
# Constants
site = "https://piugame.com/leaderboard/"
ranking_link = site + 'over_ranking.php'

In [ ]:
# Open the page
first_page = page_opener.open(ranking_link)
soup = BeautifulSoup(first_page, 'html.parser')

# Get last page
last_page = None
icons = soup.find_all("button", class_="icon")
for icon in icons:
    page = int(icon['onclick'].split("page=")[1][:-1])
    if "last" in icon.findChildren()[0]['class']:
        last_page = page
print(last_page)

In [ ]:
# helper
def get_song_title(song_soup):
    return song_soup.find_all("div", class_="songName_w")[0].findChildren("p")[0].text

def get_song_level(song_soup):
    level = ""
    
    song_lvl_img_collection = song_soup.find_all("div", class_="stepBall_img_wrap")[0].find_all("img")
    for img in song_lvl_img_collection:
        text_to_analyse = img['src']

        # check step type
        if "s_text" in text_to_analyse:
            level += "S"
        elif "d_text" in text_to_analyse:
            level += "D"
        elif "c_text" in text_to_analyse:
            level += "C"

        # check level
        if "_num_" in text_to_analyse:
            # format is "_num_2.png", get the element after _num_ and the first character only
            level += text_to_analyse.split("_num_")[1][0]
    
    return level

In [ ]:
result = {}

# Loop from current page until last page
for index in range(1, last_page + 1):
    print("current page: ", str(index))
    url_to_access = ranking_link + "?&&page=" + str(index)
    
    curr_page = page_opener.open(url_to_access)
    curr_soup = BeautifulSoup(curr_page, 'html.parser')
    
    # assume exactly 1 list
    div_with_song_list = curr_soup.find_all("ul", "rating_ranking_list")[0]
    
    # loop through all songs in a page
    items = div_with_song_list.find_all("li")
    for item in items:
        song_link = item.findChildren("a")[0]['href']
        
        complete_song_url = site + song_link
        song_page = page_opener.open(complete_song_url)
        song_soup = BeautifulSoup(song_page, 'html.parser')
        
        # get song title and level
        song_name = get_song_title(song_soup)
        level = get_song_level(song_soup)
        
        # get all scores
        scores = []
        score_divs = song_soup.find_all("div", class_="score")
        for score in score_divs:
            str_score = score.findChildren("i")[0].string
            scores.append(int(str_score.replace(',', '')))
        
        # put into result
        to_insert = {"title": song_name, "total": sum(scores), "n": len(scores)}
        if level not in result:
            result[level] = [to_insert]
        else:
            result[level].append(to_insert)

In [ ]:
# Dump result into a json file
with open("final_data.json", "w") as outfile:
    json.dump(result, outfile)

In [ ]:
# Process into csv
with open("final_data.json", "r", encoding="utf8") as file:
    data = json.load(file)
    
    for key in data:
        songs = data[key]
        sorted_songs = sorted(songs, key=lambda x: -1 * (x['total']/x['n']))
        
        header = ['title', 'total', 'n', 'average']
        result_csv = [header]
        for song in sorted_songs:
            result_csv.append([song['title'], song['total'], song['n'], song['total']/song['n']])
            
        with open(key + ".csv", "w", encoding="utf8", newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(result_csv)